# 使用sagemaker-processing-job 进行视频文件预处理

In [124]:
import sagemaker
from sagemaker.tensorflow import TensorFlow
import os 
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()
image_uri = '517141035927.dkr.ecr.us-west-2.amazonaws.com/cv-pre-processing:latest'


In [123]:
from sagemaker.processing import Processor, ScriptProcessor, ProcessingInput, ProcessingOutput

# processor = ScriptProcessor(command=['python3'],
#                 image_uri=image_uri,
#                 role=role,
#                 instance_count=1,
#                 instance_type='ml.m5.xlarge')
# processor.run(code='processing_script.py',inputs=[ProcessingInput(
#                         source='s3://sagemaker-us-west-2-517141035927/dataset/videos/',
#                         destination='/opt/ml/processing/input_data')],
#                     outputs=[ProcessingOutput(
#                         source='/opt/ml/processing/processed_data',
#                         destination='s3://sagemaker-us-west-2-517141035927/dataset/np/')],
#                     )
processor = Processor(image_uri=image_uri,
                     role=role,
                     instance_count=1,
                     instance_type="ml.c5.2xlarge")
processor.run(inputs=[ProcessingInput(
                        source='s3://sagemaker-us-west-2-517141035927/dataset/videos/',
                        destination='/opt/ml/processing/input_data')],
#                         destination='/opt/ml/processing/input_data',s3_input_mode = 'Pipe')],
                    outputs=[ProcessingOutput(
                        source='/opt/ml/processing/processed_data',
                        destination='s3://sagemaker-us-west-2-517141035927/dataset/np/')],
                    )


Job Name:  cv-pre-processing-2020-12-28-04-38-15-019
Inputs:  [{'InputName': 'input-1', 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-517141035927/dataset/videos/', 'LocalPath': '/opt/ml/processing/input_data', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'output-1', 'S3Output': {'S3Uri': 's3://sagemaker-us-west-2-517141035927/dataset/np/', 'LocalPath': '/opt/ml/processing/processed_data', 'S3UploadMode': 'EndOfJob'}}]
.....................
2020-12-28 04:41:31.961475: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2020-12-28 04:41:31.961511: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
filename is /opt/ml/processing/input_data/Drama_1.mp4
filen

## 使用多实例并行预处理

In [129]:
processor = Processor(image_uri=image_uri,
                     role=role,
                     instance_count=3,
                     instance_type="ml.c5.2xlarge")

processor.run(inputs=[ProcessingInput(
                        source='s3://sagemaker-us-west-2-517141035927/dataset/videos/',
                        destination='/opt/ml/processing/input_data',
                        s3_data_distribution_type='ShardedByS3Key')],
#                         destination='/opt/ml/processing/input_data',s3_input_mode = 'Pipe')],
                    outputs=[ProcessingOutput(
                        source='/opt/ml/processing/processed_data',
                        destination='s3://sagemaker-us-west-2-517141035927/dataset/np/')],
                    )


Job Name:  cv-pre-processing-2021-01-06-07-29-46-797
Inputs:  [{'InputName': 'input-1', 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-517141035927/dataset/videos/', 'LocalPath': '/opt/ml/processing/input_data', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'ShardedByS3Key', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'output-1', 'S3Output': {'S3Uri': 's3://sagemaker-us-west-2-517141035927/dataset/np/', 'LocalPath': '/opt/ml/processing/processed_data', 'S3UploadMode': 'EndOfJob'}}]
.......................2021-01-06 07:33:30.793915: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-01-06 07:33:30.793946: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-01-06 07:33:30.869076: W tensorflow/stream_executor/pl